In [ ]:
import aes2_added_fb_prize_as_features_preprocessing

In [ ]:
import gc

gc.collect()

In [ ]:
import pickle

with open("/kaggle/input/train_data/train_feats.pickle", "rb") as f:
    train_feats = pickle.load(f)
with open("/kaggle/input/aes2_train_data/X.pickle", "rb") as f:
    X = pickle.load(f)
with open("/kaggle/input/train_data/y.pickle", "rb") as f:
    y = pickle.load(f)
with open("/kaggle/input/train_data/y_split.pickle", "rb") as f:
    y_split = pickle.load(f)
with open(
    "/kaggle/input/train_data/feature_select.pickle", "rb"
) as f:
    feature_select = pickle.load(f)
    
# aes2_added_fb_prize_as_features_preprocessing.feature_select = feature_select

In [ ]:
train_feats.iloc[:5, -8:]

In [ ]:
import numpy as np

def clean_feature_names(features):
    illegal_chars = ['[', ']', '<', '>']
    cleaned_features = []
    for feature in features:
        for char in illegal_chars:
            feature = feature.replace(char, 'lessthan')
        cleaned_features.append(feature)
    return cleaned_features

train_feats.columns = clean_feature_names(train_feats.columns)
feature_select = clean_feature_names(feature_select)
X = train_feats[feature_select].astype(np.float32).values

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, cohen_kappa_score
from aes2_added_fb_prize_as_features_preprocessing import *

n_splits = 15
models = []
predictions = []
f1_scores = []
kappa_scores = []


class Predictor:
    def __init__(self, models: list):
        self.models = models
#         self.xgb_boost_best_iter = models[1].
    def predict(self, X):
        n_models = len(self.models)
        predicted = None
        n = 0.749
        for i, model in enumerate(self.models):
            if i == 0:
                predicted = n*model.predict(X)
            else:
                if not isinstance(X, xgb.DMatrix):
                    X = xgb.DMatrix(X)
                predicted += (1-n)*model.predict(X)
        return predicted

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
xgboost_best_iters = []
light_best_iters = []
LOAD = False # re-train
if not LOAD:
    for i in range(n_splits):
        random_indices = np.random.choice(X.shape[0], size=5000, replace=False)
        random_test_X = X[random_indices, :]
        random_test_y = y_split[random_indices]
        lgb_predictor = lgb.Booster(model_file=f'/kaggle/input/3models/aes-lgbm/fold_{i+1}.txt')
        # cat_model = CatBoostRegressor()
        # cat_model.load_model(f'kaggle/input/aes-catboost/fold_{i+1}.cbm')
        xgb_regressor = xgb.Booster()
        xgb_regressor.load_model(f'kaggle/input/3models/aes-xgboost/fold_{i+1}.bin')

        predictor = Predictor([lgb_predictor, xgb_regressor])
        
        models.append(predictor)
        
        # # Make predictions on the test data for this fold
        #         # Make predictions on the test data for this fold
        predictions_fold = predictor.predict(random_test_X)
        predictions_fold = predictions_fold + a
        predictions_fold = predictions_fold.clip(1, 6).round().astype(int)
        
        # # Confusion Matrix for each fold
        # cm = confusion_matrix(random_test_y, predictions_fold, labels=[x for x in range(1,7)])
        # disp = ConfusionMatrixDisplay(confusion_matrix=cm,
        #                               display_labels=[x for x in range(1,7)])
        # disp.plot()
        # plt.show()
        
        # # Calculate and store the F1 score for this fold
        f1_fold = f1_score(random_test_y, predictions_fold, average='weighted')
        f1_scores.append(f1_fold)

        # Calculate and store the Cohen's kappa score for this fold
        kappa_fold = cohen_kappa_score(random_test_y, predictions_fold, weights='quadratic')
        kappa_scores.append(kappa_fold)

        print(f'F1 score across fold: {f1_fold}')
        print(f'Cohen kappa score across fold: {kappa_fold}')
        gc.collect()
else:
    for i, (train_index, test_index) in enumerate(skf.split(X, y_split), 1):
    # Split the data into training and testing sets for this fold
        print('fold',i)
        X_train_fold, X_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold, y_test_fold_int = y[train_index], y[test_index], y_split[test_index]
        callbacks = [log_evaluation(period=25), early_stopping(stopping_rounds=75,first_metric_only=True)]
        light = lgb.LGBMRegressor(
                objective = qwk_obj,
                metrics = 'None',
                learning_rate = 0.05,
                max_depth = 8,
                num_leaves = 10,
                colsample_bytree=0.3,
                reg_alpha = 0.7,
                reg_lambda = 0.1,
                n_estimators=700,
                random_state=42,
                extra_trees=True,
                class_weight='balanced',
                # device='gpu' if CUDA_AVAILABLE else 'cpu',
                verbosity = - 1
            )

        # Fit the model on the training data for this fold  
        light.fit(
            X_train_fold,
            y_train_fold,
            eval_names=['train', 'valid'],
            eval_set=[(X_train_fold, y_train_fold), (X_test_fold, y_test_fold)],
            eval_metric=quadratic_weighted_kappa,
            callbacks=callbacks
        )
        light_best_iters.append(light.best_iteration_)
        
        
        light.booster_.save_model(f'kaggle/out/aes-lgbm/fold_{i}.txt')
        print('\nFold_{} LightGBM Model saved.\n'.format(i))
        
        
        xgb_callbacks = [
            xgb.callback.EvaluationMonitor(period=25),
            xgb.callback.EarlyStopping(75, metric_name="QWK", maximize=True, save_best=True)
        ]
        xgb_regressor = xgb.XGBRegressor(
            objective = qwk_obj,
            metrics = 'None',
            learning_rate = 0.1,
            max_depth = 8,
            num_leaves = 10,
            colsample_bytree=0.5,
            reg_alpha = 0.1,
            reg_lambda = 0.8,
            n_estimators=1024,
            random_state=42,
            extra_trees=True,
            class_weight='balanced',
            tree_method="gpu_hist",
            # device="gpu" if CUDA_AVAILABLE else "cpu",
            gpu_id = 7
        #             device='gpu',
        #             verbosity = 1
        )
        
        xgb_callbacks = [
            xgb.callback.EvaluationMonitor(period=25),
            xgb.callback.EarlyStopping(75, metric_name="QWK", maximize=True, save_best=True)
        ]
        xgb_regressor.fit(
            X_train_fold,
            y_train_fold,
            eval_set=[(X_train_fold, y_train_fold), (X_test_fold, y_test_fold)],
            eval_metric=quadratic_weighted_kappa,
            callbacks=xgb_callbacks
        )
        xgboost_best_iters.append(xgb_regressor.get_booster().best_iteration)
        
        xgb_regressor.save_model(f'kaggle/out/aes-xgboost/fold_{i}.bin')
        print('\nFold_{} XGBoost Model saved.\n'.format(i)) 
        
        predictor = Predictor([light, xgb_regressor])

        models.append(predictor)
        # Make predictions on the test data for this fold
        predictions_fold = predictor.predict(X_test_fold)
        predictions_fold = predictions_fold + a
        predictions_fold = predictions_fold.clip(1, 6).round()
        predictions.append(predictions_fold)
        # Calculate and store the F1 score for this fold
        f1_fold = f1_score(y_test_fold_int, predictions_fold, average='weighted')
        f1_scores.append(f1_fold)

        # Calculate and store the Cohen's kappa score for this fold
        kappa_fold = cohen_kappa_score(y_test_fold_int, predictions_fold, weights='quadratic')
        kappa_scores.append(kappa_fold)
    #         predictor.booster_.save_model(f'fold_{i}.txt')
        cm = confusion_matrix(y_test_fold_int, predictions_fold, labels=[x for x in range(1,7)])

        disp = ConfusionMatrixDisplay(
            confusion_matrix=cm,
            display_labels=[x for x in range(1,7)]
        )
        disp.plot()
        plt.show()
        print(f'F1 score across fold: {f1_fold}')
        print(f'Cohen kappa score across fold: {kappa_fold}')

        gc.collect()
        #if ENABLE_DONT_WASTE_YOUR_RUN_TIME:
        #    break


In [ ]:
mean_f1_score = np.mean(f1_scores)
mean_kappa_score = np.mean(kappa_scores)
# Print the mean scores
print(f'Mean F1 score across {n_splits} folds: {mean_f1_score}')
print(f'Mean Cohen kappa score across {n_splits} folds: {mean_kappa_score}')
# print(f"XGBoost mean best iters: {sum(xgboost_best_iters)/len(xgboost_best_iters)}")
# print(f"LightBoost mean best iters: {sum(light_best_iters)/len(light_best_iters)}")

In [ ]:

# from aes2_added_fb_prize_as_features_preprocessing import preprocess_test, infer
import aes2_added_fb_prize_as_features_preprocessing

test_feats = aes2_added_fb_prize_as_features_preprocessing.preprocess_test()
test_feats.columns = clean_feature_names(test_feats.columns)
aes2_added_fb_prize_as_features_preprocessing.infer(test_feats, models)
